In [14]:
import tensorflow as tf
from keras.datasets import cifar10
import sys, os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
print(tf.__version__)

2.10.0


In [6]:
sys.path.append("/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/modules")
sys.path.append("/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/modules/logifold")


In [11]:
import TargetTree, logifoldv1_5_in_progress

In [15]:
dropbox_path = "/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/"
base_models_path = dropbox_path + 'Base_Models/'
adversarial_examples_path = dropbox_path + 'Adversarial_Examples/'
adversarial_models_path = dropbox_path + 'Adversarial_Models/'

test_logifold_path = dropbox_path + 'testfolder/logifold_test/202505/'


In [16]:
(x, y), (x_test, y_test)= cifar10.load_data()
x_tr, x_v, y_tr, y_v = train_test_split(x, y, test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
x_train, x_val, y_train, y_val = x_tr,x_v,y_tr,y_v
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255
print('original dataset has been loaded.')


original dataset has been loaded.


In [17]:
import logifoldv1_4 as old_logifold


In [19]:
from keras.utils import to_categorical

In [20]:
adv_x_tr = np.load(adversarial_examples_path + 'by_ResNet/' + "pgd_0.376_x_untarget.npy")
adv_x_val = np.load(adversarial_examples_path + 'by_ResNet/' + "pgd_0.376_x_val_untarget.npy")
adv_x_test = np.load(adversarial_examples_path + 'by_ResNet/'+"pgd_0.376_x_test_untarget.npy")

training_y_long=np.concatenate([y_train,y_train+10],axis=0)
training_y_long = to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = to_categorical(validating_y_long,20)
testing_y_long=np.concatenate([y_test,y_test+10],axis=0)
testing_y_long = to_categorical(testing_y_long,20)

In [21]:
training_x = np.concatenate([x_tr,adv_x_tr],axis=0)
validating_x = np.concatenate([x_v,adv_x_val],axis=0)

In [32]:
lgfd = old_logifold.Logifold(20,
                        name = "test_old_logifold",
                        x_tr = training_x,
                        y_tr = training_y_long,
                        x_v = validating_x,
                        y_v = validating_y_long, 
                        path = test_logifold_path+'experiment1/',
                        storyFile='story_0504')

In [33]:
lgfd_new = logifoldv1_5_in_progress.Logifold(20,
                        name = "test_new_logifold",
                        x_tr = training_x,
                        y_tr = training_y_long,
                        x_v = validating_x,
                        y_v = validating_y_long, 
                        path = test_logifold_path+'experiment1_1/',
                        storyFile='story_0504_1')

In [26]:
from keras.models import load_model

In [27]:
base_model = load_model(base_models_path+'CIFAR10models/ResNet/n_9_v1_cifar10.keras')

2025-05-04 16:03:37.181477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
lgfd.add(base_model,(0,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet56')

Computing fuzzy domains...
The fuzzy domains have been computed.


In [35]:
lgfd_new.add(base_model,(0,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet56')

Computing fuzzy domains...The fuzzy domains have been computed.


In [30]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
#     print('Learning rate: ', lr)
    return lr

In [36]:
lgfd.turnSpecialist((0,),(0,0),newTarget = [[n] for n in range(10,20)],lr_schedule=lr_schedule, description = 'Specilaized from (0,) = ResNet56v1',verbose = 1)

Epoch 1/30
1250/1250 [==============================] - ETA: 0s - loss: 0.8495 - accuracy: 0.7510
Epoch 1: val_accuracy improved from -inf to 0.79370, saving model to /Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/testfolder/logifold_test/202505/experiment1/model_000_000.keras
1250/1250 [==============================] - 484s 385ms/step - loss: 0.8495 - accuracy: 0.7510 - val_loss: 0.7558 - val_accuracy: 0.7937 - lr: 0.0010
Epoch 2/30
 637/1250 [==============>...............] - ETA: 3:34 - loss: 0.5371 - accuracy: 0.8692

KeyboardInterrupt: 

In [ ]:
lgfd_new.turnSpecialist((0,),(0,0),newTarget = [[n] for n in range(10,20)],lr_schedule=lr_schedule, description = 'Specilaized from (0,) = ResNet56v1',verbose = 1)

In [ ]:
lgfd.turnSpecialist((0,),(0,0,0),newTarget = [[n for n in range(10)],[n for n in range(10,20)]],lr_schedule=lr_schedule, description = 'Specilaized from (0,) = ResNet56v1')

In [37]:
text = 'abcd'
text.removesuffix('d')

'abc'

In [39]:
text.replace('b','')

'acd'

In [40]:
text

'abcd'